In [ ]:

#@title نصب کتابخانه ها

import base64

encoded_credit = "TWFkZSBieSA6IGFpb2xkZW4="
decoded_credit = base64.b64decode(encoded_credit.encode()).decode()
print(decoded_credit)
print("=" * 30)

!pip install edge_tts
!pip install pysrt
!pip install pydub
!apt-get install -y rubberband-cli > /dev/null 2>&1
!sudo apt update > /dev/null 2>&1 && sudo apt install ffmpeg -y > /dev/null 2>&1

In [ ]:

#@title آپلود زیرنویس

from google.colab import files
import os

print("لطفاً فایل زیرنویس ترجمه‌شده خود را با فرمت .srt آپلود کنید:")
uploaded = files.upload()

if uploaded:
    subtitle_file = next(iter(uploaded.keys()))
    os.rename(subtitle_file, 'audio_fa.srt')
    print("✅ زیرنویس با موفقیت آپلود و آماده شد: audio_fa.srt")
else:
    print("❌ هیچ فایلی آپلود نشد. لطفاً دوباره تلاش کنید.")

In [ ]:

#@title ساخت سگمنت‌های صوتی با زمان‌بندی دقیق

import edge_tts
import asyncio
import pysrt
import os
import subprocess
import traceback
from pydub import AudioSegment

os.makedirs('dubbing_project/dubbed_segments', exist_ok=True)

# انتخاب گوینده
voice_choice = "Guy (EN)" #@param ["فرید (FA)", "دلارا (FA)", "سالما (AR-EG)", "شاکر (AR-EG)", "زاریه (AR-SA)", "حامد (AR-SA)", "امل (TR)", "احمد (TR)", "Jenny (EN)", "Guy (EN)", "Katja (DE)", "Conrad (DE)", "Elvira (ES)", "Alvaro (ES)", "Denise (FR)", "Henri (FR)", "Nanami (JA)", "Keita (JA)", "SunHi (KO)", "InJoon (KO)", "Xiaoxiao (ZH)", "Yunyang (ZH)", "Svetlana (RU)", "Dmitry (RU)", "Isabella (IT)", "Diego (IT)"]

# نقشه گوینده‌ها (به‌روز شده)
VOICE_MAP = {
    "فرید (FA)": "fa-IR-FaridNeural",
    "دلارا (FA)": "fa-IR-DilaraNeural",
    "سالما (AR-EG)": "ar-EG-SalmaNeural",
    "شاکر (AR-EG)": "ar-EG-ShakirNeural",
    "زاریه (AR-SA)": "ar-SA-ZariyahNeural",
    "حامد (AR-SA)": "ar-SA-HamedNeural",
    "امل (TR)": "tr-TR-EmelNeural",
    "احمد (TR)": "tr-TR-AhmetNeural",
    "Jenny (EN)": "en-US-JennyNeural",
    "Guy (EN)": "en-US-GuyNeural",
    "Katja (DE)": "de-DE-KatjaNeural",
    "Conrad (DE)": "de-DE-ConradNeural",
    "Elvira (ES)": "es-ES-ElviraNeural",
    "Alvaro (ES)": "es-ES-AlvaroNeural",
    "Denise (FR)": "fr-FR-DeniseNeural",
    "Henri (FR)": "fr-FR-HenriNeural",
    "Nanami (JA)": "ja-JP-NanamiNeural",
    "Keita (JA)": "ja-JP-KeitaNeural",
    "SunHi (KO)": "ko-KR-SunHiNeural",
    "InJoon (KO)": "ko-KR-InJoonNeural",
    "Xiaoxiao (ZH)": "zh-CN-XiaoxiaoNeural",
    "Yunyang (ZH)": "zh-CN-YunyangNeural",
    "Svetlana (RU)": "ru-RU-SvetlanaNeural",
    "Dmitry (RU)": "ru-RU-DmitryNeural",
    "Isabella (IT)": "it-IT-IsabellaNeural",
    "Diego (IT)": "it-IT-DiegoNeural",
}

voice_code = voice_choice.split("(")[1].split(")")[0] if "(" in voice_choice else "FA"

async def generate_speech():
    try:
        subprocess.run(['apt-get', 'install', '-y', 'rubberband-cli'], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except:
        pass

    try:
        subs = pysrt.open('/content/audio_fa.srt', encoding='utf-8')
    except UnicodeDecodeError:
        subs = pysrt.open('/content/audio_fa.srt', encoding='latin-1')
    except Exception as e:
        print(f"خطا در خواندن فایل زیرنویس: {str(e)}")
        return

    selected_voice = VOICE_MAP.get(voice_choice)
    if not selected_voice:
        print(f"گوینده انتخاب شده '{voice_choice}' موجود نیست.")
        return

    print(f"شروع تولید سگمنت‌های صوتی با صدای {voice_choice} ({selected_voice})...")

    with open('dubbing_project/segments_log.txt', 'w', encoding='utf-8') as log_file:
        log_file.write("=== گزارش وضعیت سگمنت‌های صوتی ===\\n\\n")

        for i, sub in enumerate(subs):
            try:
                log_file.write(f"سگمنت {i+1}:\\n")
                log_file.write(f"  متن: {sub.text}\\n")
                log_file.write(f"  زمان شروع: {sub.start}\\n")
                log_file.write(f"  زمان پایان: {sub.end}\\n")

                start_time = sub.start.to_time()
                end_time = sub.end.to_time()
                target_duration = (end_time.hour * 3600 + end_time.minute * 60 + end_time.second + end_time.microsecond / 1000000) - \
                                  (start_time.hour * 3600 + start_time.minute * 60 + start_time.second + start_time.microsecond / 1000000)

                if target_duration <= 0:
                    target_duration = 1.0

                log_file.write(f"  مدت زمان هدف: {target_duration:.2f}s\\n")

                communicate = edge_tts.Communicate(sub.text, selected_voice)
                await communicate.save(f"dubbing_project/dubbed_segments/temp_{i+1}.mp3")

                subprocess.run([
                    'ffmpeg', '-i', f"dubbing_project/dubbed_segments/temp_{i+1}.mp3",
                    '-acodec', 'pcm_s16le', '-ar', '44100', '-ac', '2',
                    '-y', f"dubbing_project/dubbed_segments/temp_wav_{i+1}.wav"
                ], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

                sound = AudioSegment.from_file(f"dubbing_project/dubbed_segments/temp_wav_{i+1}.wav")
                original_duration = len(sound) / 1000.0
                speed_factor = original_duration / target_duration
                speed_factor = max(0.5, min(speed_factor, 2.0))

                try:
                    subprocess.run([
                        'ffmpeg', '-i', f"dubbing_project/dubbed_segments/temp_wav_{i+1}.wav",
                        '-filter:a', f'rubberband=tempo={speed_factor}',
                        '-y', f"dubbing_project/dubbed_segments/dub_{i+1}.wav"
                    ], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
                except:
                    subprocess.run([
                        'ffmpeg', '-i', f"dubbing_project/dubbed_segments/temp_wav_{i+1}.wav",
                        '-filter:a', f'atempo={speed_factor}',
                        '-y', f"dubbing_project/dubbed_segments/dub_{i+1}.wav"
                    ], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

                if not os.path.exists(f"dubbing_project/dubbed_segments/dub_{i+1}.wav"):
                    silent_audio = AudioSegment.silent(duration=int(target_duration * 1000))
                    silent_audio.export(f"dubbing_project/dubbed_segments/dub_{i+1}.wav", format="wav")

                # پاکسازی موقت
                for temp in [f"temp_{i+1}.mp3", f"temp_wav_{i+1}.wav"]:
                    temp_path = f"dubbing_project/dubbed_segments/{temp}"
                    if os.path.exists(temp_path):
                        os.remove(temp_path)

                print(f"بخش {i+1} از {len(subs)} ساخته شد")

            except Exception as e:
                log_file.write(f"خطای کلی در سگمنت {i+1}: {str(e)}\\n{traceback.format_exc()}\\n")
                silent_audio = AudioSegment.silent(duration=1000)
                silent_audio.export(f"dubbing_project/dubbed_segments/dub_{i+1}.wav", format="wav")
                print(f"خطا در بخش {i+1} — فایل سکوت جایگزین شد")

    print(f"تمام سگمنت‌های صوتی با صدای {voice_choice} ساخته شدند!")

!apt-get install -y rubberband-cli > /dev/null 2>&1
await generate_speech()

In [ ]:

#@title ادغام سگمنت‌ها به فایل صوتی نهایی

import pysrt
import os
from pydub import AudioSegment
from google.colab import files

if not os.path.exists('audio_fa.srt'):
    print("❌ خطا: فایل زیرنویس 'audio_fa.srt' یافت نشد!")
else:
    print("✅ فایل زیرنویس یافت شد")

segment_dir = "dubbing_project/dubbed_segments"
if not os.path.exists(segment_dir):
    print("❌ خطا: پوشه سگمنت‌های صوتی یافت نشد!")
else:
    segments = [f for f in os.listdir(segment_dir) if f.startswith("dub_") and f.endswith(".wav")]
    print(f"✅ {len(segments)} سگمنت صوتی یافت شد")

try:
    subs = pysrt.open('audio_fa.srt', encoding='utf-8')
except UnicodeDecodeError:
    subs = pysrt.open('audio_fa.srt', encoding='latin-1')
except Exception as e:
    print(f"❌ خطا در خواندن SRT: {str(e)}")

# استفاده از همان voice_code که در سلول ساخت سگمنت تعریف شد
# (اگر به هر دلیلی تعریف نشده باشه، پیش‌فرض FA)
try:
    voice_code
except NameError:
    voice_code = "FA"

output_audio_filename = f'final_dubbed_audio_{voice_code.upper()}.mp3'

# محاسبه طول کل
last_sub = subs[-1]
total_duration_ms = (last_sub.end.hours * 3600000 + last_sub.end.minutes * 60000 +
                     last_sub.end.seconds * 1000 + last_sub.end.milliseconds)

final_audio = AudioSegment.silent(duration=total_duration_ms)

for i, sub in enumerate(subs):
    segment_path = f"{segment_dir}/dub_{i+1}.wav"
    if os.path.exists(segment_path):
        try:
            segment_audio = AudioSegment.from_file(segment_path)
            start_ms = (sub.start.hours * 3600000 + sub.start.minutes * 60000 +
                        sub.start.seconds * 1000 + sub.start.milliseconds)
            final_audio = final_audio.overlay(segment_audio, position=start_ms)
            print(f"✅ سگمنت {i+1} اضافه شد")
        except Exception as e:
            print(f"⚠️ خطا در سگمنت {i+1}: {str(e)}")

final_audio.export(output_audio_filename, format="mp3", bitrate="192k")
print(f"\n🎉 فایل صوتی نهایی ساخته شد: {output_audio_filename}")
files.download(output_audio_filename)

In [ ]:
#@title پاکسازی فایل‌های جلسه قبلی
!rm -rf /content/*